In [30]:
import pandas as pd
import numpy as np
import json

In [31]:
#from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string



"""from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer"""

'from wordcloud import WordCloud, STOPWORDS\nfrom PIL import Image\nfrom nltk.sentiment.vader import SentimentIntensityAnalyzer\nfrom langdetect import detect\nfrom nltk.stem import SnowballStemmer\nfrom nltk.sentiment.vader import SentimentIntensityAnalyzer\nfrom sklearn.feature_extraction.text import CountVectorizer'

In [32]:
#einlesen der Daten
data = pd.read_json("C:/Users/Kilian/Notebook/Bot/ML_4Semster/cryptodoge.json", lines=True)
df=data

In [33]:
#untersuchen der Größe des Datensatzes

print(df.shape)

(1007, 36)


In [34]:
#veranschaulischen der Spalten mehr Informationen dazu unter https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet
for y in df.columns.tolist():
    print(y)

created_at
id
id_str
text
source
truncated
in_reply_to_status_id
in_reply_to_status_id_str
in_reply_to_user_id
in_reply_to_user_id_str
in_reply_to_screen_name
user
geo
coordinates
place
contributors
retweeted_status
is_quote_status
quote_count
reply_count
retweet_count
favorite_count
entities
favorited
retweeted
filter_level
lang
timestamp_ms
display_text_range
extended_tweet
possibly_sensitive
quoted_status_id
quoted_status_id_str
quoted_status
quoted_status_permalink
extended_entities


In [35]:
print(df.text.head())

0    RT @dailytice: 🎁 $100 IN BABYDOGE GIVEAWAY ALE...
1    RT @CoinMarketCap: Bitcoin 'Could Collapse' El...
2    @MattWallace888 Amen brother!\n#dogecoin \n#To...
3    @BitcoinAssn The most powerful DOGE of them al...
4    RT @RakuCoin: Rakucoin $RAKUC currently on ETH...
Name: text, dtype: object


In [36]:
df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,lang,timestamp_ms,display_text_range,extended_tweet,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_entities
0,2021-06-28 16:25:33+00:00,1409548535272407047,1409548535272407040,RT @dailytice: 🎁 $100 IN BABYDOGE GIVEAWAY ALE...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-06-28 16:25:33+00:00,1409548536740552712,1409548536740552704,RT @CoinMarketCap: Bitcoin 'Could Collapse' El...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:33.961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-06-28 16:25:34+00:00,1409548538988609537,1409548538988609536,@MattWallace888 Amen brother!\n#dogecoin \n#To...,"<a href=""http://twitter.com/download/android"" ...",False,1.409546e+18,1.409546e+18,8.055323e+17,8.055323e+17,...,en,2021-06-28 16:25:34.497,"[16, 51]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-06-28 16:25:34+00:00,1409548540234194947,1409548540234194944,@BitcoinAssn The most powerful DOGE of them al...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.406900e+18,1.406900e+18,1.074695e+18,1.074695e+18,...,en,2021-06-28 16:25:34.794,"[13, 140]",{'full_text': '@BitcoinAssn The most powerful ...,0.0,NaN,NaN,NaN,NaN,NaN
4,2021-06-28 16:25:34+00:00,1409548540494245896,1409548540494245888,RT @RakuCoin: Rakucoin $RAKUC currently on ETH...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:25:34.856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,2021-06-28 16:36:26+00:00,1409551275440619527,1409551275440619520,RT @JubaJeeves: I went back to microwaving hot...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:36:26.918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,2021-06-28 16:36:27+00:00,1409551275893665795,1409551275893665792,RT @RakuCoin: Rakucoin $RAKUC currently on ETH...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:36:27.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1004,2021-06-28 16:36:28+00:00,1409551282596057090,1409551282596057088,RT @flurbnb: $120 to one person in 24 hours\n\...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,...,en,2021-06-28 16:36:28.624,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1005,2021-06-28 16:36:31+00:00,1409551296525332485,1409551296525332480,@BinanceAcademy The most powerful DOGE of them...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",True,1.314213e+18,1.314213e+18,9.856861e+17,9.856861e+17,...,en,2021-06-28 16:36:31.945,"[16, 140]",{'full_text': '@BinanceAcademy The most powerf...,0.0,NaN,NaN,NaN,NaN,NaN


In [37]:
#Breeinigen der Teweets

In [38]:
#df['created_at'] =  pd.to_datetime(df['created_at'], format='%Y-%m-%d')

In [39]:
df.text.head(5)
df=data

In [42]:
#bereinigen der Tweets
remove_rt = lambda x: re.sub("(RT @\w+: )|(@\w+):"," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|(\w+:\/\/\S+)|(\n|\t|\')"," ",x)


df["text"] = df.text.map(remove_rt).map(rt)
df["text"] = df.text.str.lower()
df.text.head(10)

0     🎁 $100 in babydoge giveaway alert 🎁  💰to ente...
1     bitcoin  could collapse  el salvador s econom...
2              888 amen brother! #dogecoin  #tothemoon
3      the most powerful doge of them all is about ...
4     rakucoin $rakuc currently on eth and just lau...
5               🔰website:   🔰telegram:   🎯high farm…  
6     we’re giving away 30,250,000 $kishu #cryptone...
7     yo #dogefather   the #dogearmy and i would li...
8     here are last week s top 10 trending tokens i...
9     rakucoin $rakuc currently on eth and just lau...
Name: text, dtype: object

In [43]:
df["text"][0]

' 🎁 $100 in babydoge giveaway alert 🎁  💰to enter retweet &amp; follow! 💰 ⏱ 24 hours only!⏱   ticeans, it’s the dailytice monday gi…'

In [51]:
#bereinigen des textes von den Emojis
def deEmojify(text):
    regrex_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return regrex_pattern.sub(r'',text)

df["text"] = df.text.map(deEmojify)

df.text.head(5)

0      $100 in babydoge giveaway alert   to enter r...
1     bitcoin  could collapse  el salvador s econom...
2              888 amen brother! #dogecoin  #tothemoon
3      the most powerful doge of them all is about ...
4     rakucoin $rakuc currently on eth and just lau...
Name: text, dtype: object

In [48]:
print(df.text.head())

0     🎁 $100 in babydoge giveaway alert 🎁  💰to ente...
1     bitcoin  could collapse  el salvador s econom...
2              888 amen brother! #dogecoin  #tothemoon
3      the most powerful doge of them all is about ...
4     rakucoin $rakuc currently on eth and just lau...
Name: text, dtype: object


In [54]:
df.text[0]


'  $100 in babydoge giveaway alert   to enter retweet &amp; follow!  ⏱ 24 hours only!⏱   ticeans, it’s the dailytice monday gi…'